In [ ]:
from pyspark.streaming.kafka import KafkaUtils
from pyspark.streaming import StreamingContext
from datetime import datetime

In [ ]:
import os
os.chdir("../..")
base_dir = os.getcwd()
base_dir

In [ ]:
os.environ["PYSPARK_SUBMIT_ARGS"] = f"--packages org.apache.spark:spark-sql-kafka-0-10_2.11 org.apache.kafka:kafka-clients-2.4.0 pyspark-shell"

In [ ]:
schema_data = spark.read.json(f"file://{base_dir}/00_input/data/subkingdoms.json")
schema = schema_data.schema

In [ ]:
schema_data.take(1)

In [ ]:
TOPIC = "topic_one"
BOOTSTRAP_SERVER = "localhost:9092"

In [ ]:
data_sdf = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", BOOTSTRAP_SERVER) \
  .option("subscribe", TOPIC) \
  .option("startingOffsets", "earliest") \
  .load()
# #     .schema(dataSchema)\

In [ ]:
string_df = data_sdf.selectExpr("CAST(value AS STRING)")
string_df.printSchema()

# Start running the query that prints the running counts to the console
query = string_df \
    .writeStream \
    .format("console") \
    .start()

query.awaitTermination()